In [1]:
import numpy as np
import xarray as xr
import os
from collections import defaultdict
import fnmatch
from tqdm.autonotebook import tqdm
from xmip.postprocessing import concat_members
import gcsfs
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

/tmp/ipykernel_24076/1363005158.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Derives annual maxima from zos_ibe datasets.

In [2]:
def get_ddict_of_datasets(input_path,ssps):
    #input path must contain directories organized by model
    ddict = defaultdict(dict)
    for model_path in tqdm(fs.ls(input_path)):
        for ssp in ssps:
            fns = ['gs://'+k for k in fs.ls(model_path) if ssp in k]
            for fn in fns:
                #ds = xr.open_dataset(fn,engine='zarr',use_cftime=True,chunks={'lat':500,'lon':500,'time':200})
                ds = xr.open_dataset(fn,engine='zarr',decode_times=xr.coders.CFDatetimeCoder(use_cftime=True),chunks={'lat':500,'lon':500,'time':200})
                
                ds = ds.expand_dims(dim={'experiment_id':[str(ssp)]})
                ds.attrs['experiment_id'] = ssp
                ddict[fn] = ds
    return ddict
    
def add_years_months_to_coords(ds):
    years = []
    months = []
    for k,ts in enumerate(ds.time.values):
        try: #cant remember what this was for, is this even the right syntax?
            years.append(ts.dt.year)
            months.append(ts.dt.month)
        except:
            years.append(ts.year)
            months.append(ts.month)
    ds = ds.assign_coords({'year':('time',years),'month':('time',months)})
    return ds

def enough_numMonths_in_period(ds,period,tol=.95):
    return len(ds.time) >= tol * 12 * (1+period[-1]-period[0])

def derive_dict_of_anmax(ddict,hist,fut):
    anmax = defaultdict(dict)
    for key,ds in tqdm(ddict.items()):
        #print(key)
        try:
            ds_hist = ds.sel(time=slice(str(hist[0]),str(hist[-1])))
            ds_fut = ds.sel(time=slice(str(fut[0]),str(fut[-1])))
            if not enough_numMonths_in_period(ds_hist,hist) or not enough_numMonths_in_period(ds_fut,fut):
                print('dataset incomplete for historical and/or future periods, not including: '+key)
                continue
            anmax[key] = xr.concat((ds_hist.groupby(ds_hist.time.dt.year).max().mean(dim='year'),
                                       ds_fut.groupby(ds_fut.time.dt.year).max().mean(dim='year')
                                       ),dim='period')
            anmax[key]['period'] = ['hist','fut']
            anmax[key].attrs = ds.attrs
        except: #irregular calendar, issue with selecting years/months
            print('Cannot select years/months due to time index issues, computing these coordinates manually, for: '+key)
            ds = add_years_months_to_coords(ds)
            ds_hist = ds.where((ds.year >= hist[0])&(ds.year <= hist[1]))
            ds_fut = ds.where((ds.year >= fut[0])&(ds.year <= fut[1]))

            if not enough_numMonths_in_period(ds_hist,hist) or not enough_numMonths_in_period(ds_fut,fut):
                print('dataset incomplete for historical and/or future periods, not including: '+key)
                continue
            
            anmax[key] = xr.concat((ds_hist.groupby(ds_hist.year).max().mean(dim='year'),
                                       ds_fut.groupby(ds_fut.year).max().mean(dim='year'),
                                       ),dim='period')
            anmax[key]['period'] = ['hist','fut']
            anmax[key].attrs = ds.attrs
    return anmax

Configure the script

In [4]:
ssps = ['ssp126','ssp245','ssp370','ssp585'] #SSPs to process #(TODO: loop over multiple, streamline code!)
zos_path = 'gs://leap-persistent/timh37/CMIP6/zos_1x1'
ibe_path = 'gs://leap-persistent/timh37/CMIP6/ibe_1x1'
output_path = 'gs://leap-persistent/timh37/CMIP6/zos_ibe_1x1_annual_maxima'
hist = [1993,2022]
fut = [2071,2100]

In [5]:
zos_ddict = {}
zos_ddict = get_ddict_of_datasets(zos_path,ssps) #open datasets (grouped by model) into dictionary

ibe_ddict = {}
ibe_ddict = get_ddict_of_datasets(ibe_path,ssps) #open datasets (grouped by model) into dictionary

  0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

/tmp/ipykernel_20129/3651984063.py:31: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(fn,engine='zarr',decode_times=xr.coders.CFDatetimeCoder(use_cftime=True),chunks={'lat':500,'lon':500,'time':200})
/tmp/ipykernel_20129/3651984063.py:31: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(fn,engine='zarr',decode_times=xr.coders.CFDatetimeCoder(use_cftime=True),chunks={'lat':500,'lon':500,'time':200})
/tmp/ipykernel_20129/3651984063.py:31: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(fn,engine='zarr',decode_times=xr.code

In [6]:
zos_ibe_ddict = {}
zos_keys = list(zos_ddict.keys())
for k,v in tqdm(ibe_ddict.items()):
    zos_key = k.replace('psl','zos').replace('ibe','zos').replace('Amon','Omon').replace(k.split('.')[1],'*')
    matched_keys = fnmatch.filter(zos_keys,zos_key)
    if len(matched_keys)==1:
        v['zos_ibe'] = zos_ddict[matched_keys[0]]['zos'] + v['ibe']
        zos_ibe_ddict[k.split('/')[-1].replace('psl.','').replace('Amon.','')] = v[['zos_ibe']]

  0%|          | 0/1350 [00:00<?, ?it/s]

In [7]:
anmax = derive_dict_of_anmax(zos_ibe_ddict,hist,fut) #compute month means for each dataset
anmax = concat_members(anmax) #not sure why this also automatically combines experiment ids, but ok?

  0%|          | 0/1033 [00:00<?, ?it/s]

dataset incomplete for historical and/or future periods, not including: CESM2-WACCM.gn.r2i1p1f1.hist_ssp370.1980-2055
dataset incomplete for historical and/or future periods, not including: CESM2-WACCM.gn.r3i1p1f1.hist_ssp370.1980-2055
dataset incomplete for historical and/or future periods, not including: CNRM-CM6-1.gr.r10i1p1f2.hist_ssp245.1980-2020
dataset incomplete for historical and/or future periods, not including: CNRM-CM6-1.gr.r7i1p1f2.hist_ssp245.1980-2020
dataset incomplete for historical and/or future periods, not including: CNRM-CM6-1.gr.r8i1p1f2.hist_ssp245.1980-2020
dataset incomplete for historical and/or future periods, not including: CNRM-CM6-1.gr.r9i1p1f2.hist_ssp245.1980-2020
dataset incomplete for historical and/or future periods, not including: EC-Earth3-AerChem.gr.r4i1p1f1.hist_ssp370.1980-2055
dataset incomplete for historical and/or future periods, not including: HadGEM3-GC31-LL.gn.r11i1p1f3.hist_ssp245.1980-2020
dataset incomplete for historical and/or future 

Generate dictionary of month means per model:

In [8]:
out_ddict = defaultdict(dict)
for model in np.unique([v.source_id for v in anmax.values()]):
    datasets = []
    for k,v in anmax.items():
        if v.source_id == model:
            datasets.append(v)
    model_dataset = xr.concat(datasets,coords='different',dim='experiment_id')
    out_ddict[str(model)] = model_dataset

Store:

In [9]:
query_var = 'zos_ibe'
for key,ds in tqdm(out_ddict.items()):
    fn = ds.source_id+'_zos_ibe_anmax_hist'+str(hist[0])+'-'+str(hist[-1])+'_fut'+str(fut[0])+'-'+str(fut[-1])
    
    ds['member_id'] = ds['member_id'].astype('str')
    ds['experiment_id'] = ds['experiment_id'].astype('str')
    ds = ds.assign_coords({'source_id':ds.source_id})
    try:
        ds[query_var] = ds[query_var].chunk({'lat':500,'lon':500,'period':1,'member_id':50,'experiment_id':1})
    except:
        ds[query_var] = ds[query_var].chunk({'id':1000,'period':1,'member_id':200,'experiment_id':1})
    ds.to_zarr(os.path.join(output_path,fn),mode='w',zarr_version=2)
    ds.close()

  0%|          | 0/39 [00:00<?, ?it/s]

/tmp/ipykernel_20129/2986513807.py:12: FutureWarning: zarr_version is deprecated, use zarr_format
  ds.to_zarr(os.path.join(output_path,fn),mode='w',zarr_version=2)
/srv/conda/envs/notebook/lib/python3.12/site-packages/flox/core.py:1457: RuntimeWarning: All-NaN slice encountered
  result = combine(array, axis=axis_, keepdims=True)
/tmp/ipykernel_20129/2986513807.py:12: FutureWarning: zarr_version is deprecated, use zarr_format
  ds.to_zarr(os.path.join(output_path,fn),mode='w',zarr_version=2)
/tmp/ipykernel_20129/2986513807.py:12: FutureWarning: zarr_version is deprecated, use zarr_format
  ds.to_zarr(os.path.join(output_path,fn),mode='w',zarr_version=2)
/tmp/ipykernel_20129/2986513807.py:12: FutureWarning: zarr_version is deprecated, use zarr_format
  ds.to_zarr(os.path.join(output_path,fn),mode='w',zarr_version=2)
/tmp/ipykernel_20129/2986513807.py:12: FutureWarning: zarr_version is deprecated, use zarr_format
  ds.to_zarr(os.path.join(output_path,fn),mode='w',zarr_version=2)
/tmp/ip